<a href="https://colab.research.google.com/github/surajmnit/datascience/blob/main/01_json_normalize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q atlassian-python-api

from atlassian import Jira
from IPython.display import display, HTML, display_html
import pandas as pd

pd.set_option('max_colwidth', 140)

     |████████████████████████████████| 122kB 6.5MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


In [ ]:
jira = Jira(
    url = "https://surajtr.atlassian.net/",
    username = "suraj.royal111@gmail.com",
    password = "3AB9KexivbHZg5H9G9UK6590",
)

In [ ]:
results = jira.jql("project = LEAR", limit=5, fields=["issuetype", "status", "status", "summary"])

In [ ]:
results['issues']

In [ ]:
df = pd.DataFrame.from_records(results["issues"], columns=["key", "fields",])

In [ ]:
df.head()

,key,fields
0,LEAR-7,"{'summary': 'do it asap', 'issuetype': {'self': 'https://surajtr.atlassian.net/rest/api/2/issuetype/10003', 'id': '10003', 'description'..."
1,LEAR-6,"{'summary': 'final issue', 'issuetype': {'self': 'https://surajtr.atlassian.net/rest/api/2/issuetype/10001', 'id': '10001', 'description..."
2,LEAR-5,"{'summary': 'kcell 2018', 'issuetype': {'self': 'https://surajtr.atlassian.net/rest/api/2/issuetype/10001', 'id': '10001', 'description'..."
3,LEAR-4,"{'summary': 'tsa', 'issuetype': {'self': 'https://surajtr.atlassian.net/rest/api/2/issuetype/10001', 'id': '10001', 'description': 'Func..."
4,LEAR-3,"{'summary': 'tef peru', 'issuetype': {'self': 'https://surajtr.atlassian.net/rest/api/2/issuetype/10001', 'id': '10001', 'description': ..."


In [ ]:
df = pd.DataFrame.from_records(results["issues"], columns=["key", "fields",])

df = (
    df["fields"]
    .apply(pd.Series)
    .merge(df, left_index=True, right_index = True)
)
df.drop(columns = "fields", inplace = True)

In [ ]:
# Extract the issue type name and assign it to a new column called "issue_type"
df_issue_type = (
    df["issuetype"]
    .apply(pd.Series)
    .rename(columns={"name": "issue_type_name"})["issue_type_name"]
)
df = df.assign(issue_type_name = df_issue_type)
df.drop(columns = "issuetype", inplace = True)
df

,summary,status,key,issue_type_name
0,do it asap,"{'self': 'https://surajtr.atlassian.net/rest/api/2/status/10000', 'description': '', 'iconUrl': 'https://surajtr.atlassian.net/', 'name'...",LEAR-7,Sub-task
1,final issue,"{'self': 'https://surajtr.atlassian.net/rest/api/2/status/10000', 'description': '', 'iconUrl': 'https://surajtr.atlassian.net/', 'name'...",LEAR-6,Story
2,kcell 2018,"{'self': 'https://surajtr.atlassian.net/rest/api/2/status/10000', 'description': '', 'iconUrl': 'https://surajtr.atlassian.net/', 'name'...",LEAR-5,Story
3,tsa,"{'self': 'https://surajtr.atlassian.net/rest/api/2/status/10000', 'description': '', 'iconUrl': 'https://surajtr.atlassian.net/', 'name'...",LEAR-4,Story
4,tef peru,"{'self': 'https://surajtr.atlassian.net/rest/api/2/status/10000', 'description': '', 'iconUrl': 'https://surajtr.atlassian.net/', 'name'...",LEAR-3,Story


In [ ]:
FIELDS = ["key", "fields.summary", "fields.issuetype.name", "fields.status.name", "fields.status.statusCategory.name"]
df = pd.json_normalize(results["issues"])
df[FIELDS]

,key,fields.summary,fields.issuetype.name,fields.status.name,fields.status.statusCategory.name
0,LEAR-7,do it asap,Sub-task,Backlog,To Do
1,LEAR-6,final issue,Story,Backlog,To Do
2,LEAR-5,kcell 2018,Story,Backlog,To Do
3,LEAR-4,tsa,Story,Backlog,To Do
4,LEAR-3,tef peru,Story,Backlog,To Do


## Some Extras

In [ ]:
# Use record_path instead of passing the list contained in results["issues"]
pd.json_normalize(results, record_path="issues")[FIELDS]

,key,fields.summary,fields.issuetype.name,fields.status.name,fields.status.statusCategory.name
0,LEAR-7,do it asap,Sub-task,Backlog,To Do
1,LEAR-6,final issue,Story,Backlog,To Do
2,LEAR-5,kcell 2018,Story,Backlog,To Do
3,LEAR-4,tsa,Story,Backlog,To Do
4,LEAR-3,tef peru,Story,Backlog,To Do


In [ ]:
# Only recurse down to the second level, in this case the statusCategory name field won't be included in the resulting DataFrame.
pd.json_normalize(results, record_path="issues", max_level = 2)[FIELDS]

KeyError: ignored

In [ ]:
# Separate level prefixes with a "-" instead of the default "."
FIELDS = ["key", "fields-summary", "fields-issuetype-name", "fields-status-name", "fields-status-statusCategory-name"]
pd.json_normalize(results["issues"], sep = "-")[FIELDS]